## Import Libaries

In [0]:
# Data science tools

import pandas as pd
import numpy as np
import os

# PyTorch
import torch
# Torch and Torchvision
import torchvision 
from torch import optim, cuda
import torch.nn as nn
#from torch.utils import data
from torch.utils.data import Dataset, TensorDataset,DataLoader,sampler
#from torchvision import datasets
from torchvision import transforms, datasets, models


# Image manipulations
from PIL import Image
# Useful for examining network
from torchsummary import summary
# Timing utility
from timeit import default_timer as timer

In [2]:
# Visualizations
import matplotlib.pyplot as plt
#%matplotlib inline
plt.rcParams['font.size'] = 14
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
!pip install tensorboardX
import tensorflow as tf
import scipy.misc
from tensorboardX import SummaryWriter
%load_ext tensorboard
logs_base_dir = "Logs"
os.makedirs(logs_base_dir, exist_ok=True)
# debug ref: https://github.com/pytorch/pytorch/issues/22676

     |████████████████████████████████| 204kB 21.8MB/s 


In [4]:
# import Customed dataloader 
import DataPreparation  

trainset = DataPreparation.trainset 
dataloaders = DataPreparation.dataloaders 

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
50000 10000

(50000, 3, 32, 32)
train_mean [0.4914009  0.48215896 0.4465308 ]
train_std [0.24703279 0.24348423 0.26158753]
Filter 50% of the labels for training (bird, deer and truck
length of customed training data 42500
Files already downloaded and verified


## Build the end to end Model


In [0]:
class EndtoEndModel(nn.Module):
    def __init__(self):
        super(EndtoEndModel, self).__init__()
        
        # batch_size : 32
        # Input size to the Autoencoder : [batch_size, 3, 32, 32]
        # Output size from the Autoencoder : [batch_size, 3, 32, 32]
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 6, kernel_size = 3, stride=1, padding=1), 
            nn.ReLU(),
            nn.Conv2d(in_channels = 6, out_channels = 12, kernel_size = 3, stride=1, padding=1), 
            nn.ReLU(),
            nn.Conv2d(in_channels = 12, out_channels = 24, kernel_size = 3, stride=2, padding=1), 
            nn.ReLU(),
            nn.Conv2d(in_channels = 24, out_channels = 48, kernel_size = 3, stride=2, padding=1),
            nn.ReLU() 
            ## Output size : [batch_size, 48, 16, 16]
        )
        self.decoder = nn.Sequential(
            # For Upscaling :
            # 2x2 kernels can only learn nearest pixel upscaling.
            # 3x3 kernels can do bilinear but will require asymmetric padding.
            # But 4x4 can do bilinear again without asymmetrical padding.
          
            nn.ConvTranspose2d(in_channels =48, out_channels =24, kernel_size =4, stride=2, padding=1), 
            nn.ReLU(), 
            nn.ConvTranspose2d(in_channels =24, out_channels =12, kernel_size =4, stride=2, padding=1), 
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels =12, out_channels =6, kernel_size =3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels =6, out_channels =3, kernel_size =3, stride=1, padding=1),  
            nn.Sigmoid()
        )
        self.classifier_conv_layer = nn.Sequential(

            # xiaoya: no Relu
            nn.Conv2d(in_channels=48, out_channels=48, kernel_size=3, padding=1),
            nn.BatchNorm2d(48),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=48, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.05), # TODO experiment with 0.2/0.4
               
        )
        self.classifier_fc_layer = nn.Sequential(
            nn.Linear(512, 128),
            nn.Dropout(p=0.1),
            nn.ReLU(inplace=True),
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 10)
        )
    def forward(self, x):
        #autoencoder branch
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        # classifier branch
        # conv layers
        x = self.classifier_conv_layer(encoded)       
        # flatten)
        x = x.view(x.size(0), -1) 
        #print(x.size())      
        # fc layer
        x = self.classifier_fc_layer(x)
        return decoded,x
def toGPU(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return x

In [11]:
def weights_init(m):
    classname = m.__class__.__name__
    #torch.nn.init.xavier_uniform(conv1.weight)
    if classname.find('Conv2d') != -1:
        torch.nn.init.xavier_uniform(m.weight)
        #m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.xavier_uniform(m.weight)
        #m.weight.data.normal_(1.0, 0.02)
        #m.bias.data.fill_(0)
def create_model():
    model = EndtoEndModel()
    model = toGPU(model)
    return model
model=create_model()
summary(model, input_size=(3, 32, 32), batch_size=32, device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [32, 6, 32, 32]             168
              ReLU-2            [32, 6, 32, 32]               0
            Conv2d-3           [32, 12, 32, 32]             660
              ReLU-4           [32, 12, 32, 32]               0
            Conv2d-5           [32, 24, 16, 16]           2,616
              ReLU-6           [32, 24, 16, 16]               0
            Conv2d-7             [32, 48, 8, 8]          10,416
              ReLU-8             [32, 48, 8, 8]               0
   ConvTranspose2d-9           [32, 24, 16, 16]          18,456
             ReLU-10           [32, 24, 16, 16]               0
  ConvTranspose2d-11           [32, 12, 32, 32]           4,620
             ReLU-12           [32, 12, 32, 32]               0
  ConvTranspose2d-13            [32, 6, 32, 32]             654
             ReLU-14            [32, 6,

####mapping

In [12]:
#Mapping of Classes to Indexes

#To keep track of the predictions made by the model, 
#we create a mapping of classes to indexes and indexes to classes.
#This will let us know the actual class for a given prediction.

model.class_to_idx = trainset.class_to_idx
model.idx_to_class = {
    idx: class_
    for class_, idx in model.class_to_idx.items()
}

print(list(model.idx_to_class.items()))

[(0, 'airplane'), (1, 'automobile'), (2, 'bird'), (3, 'cat'), (4, 'deer'), (5, 'dog'), (6, 'frog'), (7, 'horse'), (8, 'ship'), (9, 'truck')]


## training

In [0]:
def train(model,
          train_loader,
          valid_loader,
          save_file_name,
          max_epochs_stop=3,
          n_epochs=20,
          print_every=1,
          weight_loss_ae =1 , 
          weight_loss_clf =1,          
          criterionAE = nn.MSELoss(),
          criterionCLF = nn.CrossEntropyLoss(),
          optimizer = optim.Adam(model.parameters(),lr=1e-3,weight_decay=1e-5) 
    ):
    """Train an end to end AutoencoderClassification Model

    Params Explained
    --------
        model (PyTorch model): cnn to train
        train_loader (PyTorch dataloader): training dataloader to iterate through
        valid_loader (PyTorch dataloader): validation dataloader used for early stopping
        save_file_name (str ending in '.pkl'): file path to save the model state dict
        max_epochs_stop (int): maximum number of epochs with no improvement in validation loss for early stopping
        n_epochs (int): maximum number of training epochs
        weight_loss_ae(int) : the weights of criterionAE in our objective to minimize the loss
        weight_loss_clf(int):  the weights of criterionCLF in our objective to minimize the lo
        print_every (int): frequency of epochs to print training stats
        criterionAE (PyTorch loss): objective to minimize for the Autoencoder branch
        criterionCLF (PyTorch loss): objective to minimize for the classifier branch
        optimizer (PyTorch optimizier): optimizer to compute gradients of model parameters
    Returns
    --------
        model (PyTorch model): trained Model 
        history (DataFrame): history of train and validation losses and accuracy
    """
    # Early stopping intialization
    epochs_no_improve = 0
    valid_loss_min = np.Inf

    valid_max_acc = 0
    history = []

    # Number of epochs already trained (if using loaded in model weights)
    try:
        print(f'Model has been trained for: {model.epochs} epochs.\n')
    except:
        model.epochs = 0
        print(f'Starting Training from Scratch.\n')

    overall_start = timer()

    # Main loop
    for epoch in range(n_epochs):

        # keep track of training and validation loss each epoch
        train_loss = 0.0
        train_AEloss = 0.0
        train_CLFloss = 0.0
        valid_loss = 0.0
        valid_AEloss = 0.0
        valid_CLFloss = 0.0
        train_acc = 0
        valid_acc = 0

        # Set to training
        model.train()
        start = timer()

        # Training loop
        for num, (data, target) in enumerate(train_loader):
            # Tensors to gpu
            data, target = toGPU(data),toGPU(target)

            # Clear gradients
            optimizer.zero_grad()
            # load model
            encoded, output = model(data)

            # Loss and backpropagation of gradients
            AEloss = criterionAE(encoded, data)

            CLFloss = criterionCLF(output,target)
            loss = weight_loss_ae*AEloss + weight_loss_clf*CLFloss
            
            loss.backward()

            # Update the parameters
            optimizer.step()

            # Track train loss by multiplying average loss by number of examples in batch
            train_loss += loss * data.size(0)
            train_AEloss += AEloss.item() * data.size(0)
            train_CLFloss += CLFloss.item() * data.size(0)
            # Calculate accuracy by finding max log probability
            _, pred = torch.max(output, dim=1)
            correct_tensor = pred.eq(target.data.view_as(pred))
            # Need to convert correct tensor from int to float to average
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            # Multiply average accuracy times the number of examples in batch
            train_acc += accuracy.item() * data.size(0)

            # Track training progress
            print(
                f'Epoch: {epoch}\t{100 * (num + 1) / len(train_loader):.2f}% complete. {timer() - start:.2f} seconds elapsed in epoch.',
                end='\r')

        # After training loops ends, start validation
        #else:
        model.epochs += 1

        # Don't need to keep track of gradients
        with torch.no_grad():
            # Set to evaluation mode
            model.eval()

            # Validation loop
            for data, target in valid_loader:
                # Tensors to gpu
                if torch.cuda.is_available():
                    data, target = data.cuda(), target.cuda()
                # Forward pass
                encoded, output = model(data)

                # Validation loss
                AEloss = criterionAE(encoded, data)

                CLFloss = criterionCLF(output,target)
                loss = weight_loss_ae*AEloss + weight_loss_clf*CLFloss
                
                # Multiply average loss times the number of examples in batch
                valid_loss += loss * data.size(0)
                valid_AEloss += AEloss.item() * data.size(0)
                valid_CLFloss += CLFloss.item() * data.size(0)
                # Calculate validation accuracy
                _, pred = torch.max(output, dim=1)
                correct_tensor = pred.eq(target.data.view_as(pred))
                accuracy = torch.mean(
                    correct_tensor.type(torch.FloatTensor))
                # Multiply average accuracy times the number of examples
                valid_acc += accuracy.item() * data.size(0)

            # Calculate average losses
            train_loss = train_loss / len(train_loader.dataset)
            valid_loss = valid_loss / len(valid_loader.dataset)
            train_AEloss = train_AEloss / len(train_loader.dataset)
            valid_AEloss = valid_AEloss / len(valid_loader.dataset)
            train_CLFloss = train_CLFloss / len(train_loader.dataset)
            valid_CLFloss = valid_CLFloss / len(valid_loader.dataset)            
            # Calculate average accuracy
            train_acc = train_acc / len(train_loader.dataset)
            valid_acc = valid_acc / len(valid_loader.dataset)

            history.append(
                [train_loss.item(), valid_loss.item(), train_acc, valid_acc,train_AEloss,valid_AEloss,train_CLFloss,valid_CLFloss]
                )

            # Print training and validation results
            if (epoch + 1) % print_every == 0:
                print(
                    f'\nEpoch: {epoch}'
                )
                print(
                    f'\t\tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}'
                    )
                print(
                    f'\t\tTraining AELoss: {train_AEloss:.4f} \tValidation AELoss: {valid_AEloss:.4f}'
                )
                print(
                    f'\t\tTraining CLFLoss: {train_CLFloss:.4f} \tValidation CLFLoss: {valid_CLFloss:.4f}'
                )
                print(
                    f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%'
                )

            # Save the model if validation loss decreases
            if valid_loss < valid_loss_min:
                # Save model
                torch.save(model.state_dict(), save_file_name)
                # Track improvement
                epochs_no_improve = 0
                valid_loss_min = valid_loss
                valid_best_acc = valid_acc
                best_epoch = epoch

            # Otherwise increment count of epochs with no improvement
            else:
                epochs_no_improve += 1
                # Trigger early stopping
                if epochs_no_improve >= max_epochs_stop:
                    print(
                        f'\nEarly Stopping! Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
                    )
                    total_time = timer() - overall_start
                    print(
                        f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.'
                    )

                    # Load the best state dict
                    model.load_state_dict(torch.load(save_file_name))
                    # Attach the optimizer
                    model.optimizer = optimizer

                    # Format history
                    history = pd.DataFrame(
                        history,
                        columns=[
                            'train_loss', 'valid_loss', 'train_acc',
                            'valid_acc','train_AEloss','valid_AEloss',
                            'train_CLFloss','valid_CLFloss'
                        ])
                    return model, history

    # Attach the optimizer
    model.optimizer = optimizer
    # Record overall time and print out stats
    total_time = timer() - overall_start
    print(
        f'\nBest epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
    )
    print(
        f'{total_time:.2f} total seconds elapsed. {total_time / (epoch):.2f} seconds per epoch.'
    )
    # Format history
    history = pd.DataFrame(
        history,
        columns=['train_loss', 'valid_loss', 'train_acc', 
                 'valid_acc','train_AEloss','valid_AEloss',
                 'train_CLFloss','valid_CLFloss'])
    return model, history

## Training history

In [20]:
model_output, history = train(
    model,
    dataloaders['train'],
    dataloaders['val'],
    save_file_name='end2en-kr.pkl',
    max_epochs_stop=30,
    n_epochs=30)

history.to_csv('kr-30-60-history.csv')

Model has been trained for: 30 epochs.


Epoch: 0
		Training Loss: 1.2073 	Validation Loss: 1.2149
		Training AELoss: 0.6632 	Validation AELoss: 0.5473
		Training CLFLoss: 0.5441 	Validation CLFLoss: 0.6676
		Training Accuracy: 80.97%	 Validation Accuracy: 77.68%

Epoch: 1
		Training Loss: 1.2041 	Validation Loss: 1.2279
		Training AELoss: 0.6630 	Validation AELoss: 0.5476
		Training CLFLoss: 0.5411 	Validation CLFLoss: 0.6804
		Training Accuracy: 81.12%	 Validation Accuracy: 77.56%

Epoch: 2
		Training Loss: 1.1901 	Validation Loss: 1.2518
		Training AELoss: 0.6636 	Validation AELoss: 0.5475
		Training CLFLoss: 0.5265 	Validation CLFLoss: 0.7043
		Training Accuracy: 81.54%	 Validation Accuracy: 76.72%

Epoch: 3
		Training Loss: 1.1942 	Validation Loss: 1.2270
		Training AELoss: 0.6638 	Validation AELoss: 0.5474
		Training CLFLoss: 0.5303 	Validation CLFLoss: 0.6796
		Training Accuracy: 81.56%	 Validation Accuracy: 77.98%

Epoch: 4
		Training Loss: 1.1839 	Validation Loss: 1.2076
		Tra

## Function to Evaluate Model Over All Classes

The next function iterates through the testing set in order to make predictions for each image. It calculates performance for each category.

In [0]:
def accuracy(output, target, topk=(1, )):
    """Compute the topk accuracy(s)"""
    if torch.cuda.is_available():
        output = output.to('cuda')
        target = target.to('cuda')

    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        # Find the predicted classes and transpose
        _, pred = output.topk(k=maxk, dim=1, largest=True, sorted=True)
        pred = pred.t()

        # Determine predictions equal to the targets
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []

        # For each k, find the percentage of correct
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size).item())
        return res

In [0]:
def evaluate(model, test_loader, criterionAE,criterionCLF, topk=(1, 3)):
    """Measure the performance of a trained PyTorch model

    Params
    --------
        model (PyTorch model): trained cnn for inference
        test_loader (PyTorch DataLoader): test dataloader
        topk (tuple of ints): accuracy to measure

    Returns
    --------
        results (DataFrame): results for each category

    """

    classes = []
    losses = []
    # Hold accuracy results
    acc_results = np.zeros((len(test_loader.dataset), len(topk)))
    i = 0

    model.eval()
    with torch.no_grad():

        # Testing loop
        for data, targets in test_loader:

            # Tensors to gpu
            if torch.cuda.is_available():
                data, targets = data.to('cuda'), targets.to('cuda')
            # Raw model output
            encoded,out = model(data)
            # Iterate through each example
            for pred, true in zip(out, targets):
                # Find topk accuracy
                acc_results[i, :] = accuracy(
                    pred.unsqueeze(0), true.unsqueeze(0), topk)
                classes.append(model.idx_to_class[true.item()])
                """
                AEloss = criterionAE(encoded, data)

                CLFloss = criterionCLF(output,target)
                loss = weight_loss_ae*AEloss + weight_loss_clf*CLFloss
                """
                # Calculate the loss
                loss = criterionCLF(pred.view(1, 10), true.view(1))
                losses.append(loss)
                i += 1

    # Send results to a dataframe and calculate average across classes
    results = pd.DataFrame(acc_results, columns=[f'top{i}' for i in topk])
    results['class'] = classes
    results['loss'] = losses
    results = results.groupby(classes).mean()

    return results.reset_index().rename(columns={'index': 'class'})

In [0]:
criterionAE = nn.MSELoss() 
criterionCLF = nn.CrossEntropyLoss() 
# Evaluate the model on all the training data
results = evaluate(model_output,dataloaders['test'], criterionAE,criterionCLF)


In [24]:
#results.to_csv('kr-test-60.csv')
results

,class,top1,top3
0,airplane,85.979381,96.082474
1,automobile,90.573770,98.565574
2,bird,50.976562,78.125000
3,cat,63.257576,96.022727
4,deer,69.348659,92.337165
5,dog,81.027668,98.616601
6,frog,89.390519,97.065463
7,horse,88.977956,97.995992
8,ship,91.748527,97.642436
9,truck,79.724409,94.881890
